# Advent of Code 2021 - Day 17

In [300]:
data class Target(val horizontal: IntRange, val vertical: IntRange)

data class OrderedPair(val x: Int, val y: Int)
data class Instant(val point: OrderedPair, val velocity: OrderedPair) {
    constructor(point: Pair<Int, Int>, velocity: Pair<Int, Int>) : 
        this(OrderedPair(point.first, point.second), OrderedPair(velocity.first, velocity.second))
}

In [301]:
import java.io.File

val regex = Regex("""x=(-?\d+)..(-?\d+), y=(-?\d+)..(-?\d+)""")

val targets: List<Target> = File("Day17.input.txt")
    .bufferedReader()
    .lineSequence()
    .map(regex::find)
    .map { it!!.destructured.toList() }
    .map { it.map(String::toInt) }
    .map { (x1,x2,y1,y2) -> Target(horizontal = x1..x2, vertical = y1..y2) }
    .toList()

## Part 1

Find the highest `Instant::point` of any starting `Instant::velocity` from a starting point of 0,0 that hits the `target`.

In [302]:
fun Instant.next(drag: OrderedPair = OrderedPair(1,1)) = Instant(
    point = OrderedPair((point.x + velocity.x), (point.y + velocity.y)),
    velocity = OrderedPair((if (velocity.x > 0) velocity.x - drag.x 
        else if (velocity.x < 0) velocity.x + drag.x
        else 0), 
        (velocity.y - drag.y))
)

fun Instant.towards(target: Target, drag: OrderedPair = OrderedPair(1,1)): Boolean? {
    if (point.x in target.horizontal && point.y in target.vertical) return null

    val horizAhead = point.x < target.horizontal.start && velocity.x > 0
    val horizBehind = point.x > target.horizontal.endInclusive && velocity.x < 0
    val vertBelow = point.y < target.vertical.start && velocity.y > 0
    val vertAbove = point.y > target.vertical.endInclusive && (velocity.y < 0 || drag.y > 0)

    return (horizAhead || horizBehind) || (vertAbove || vertBelow)
}

fun Instant.hits(target: Target, drag: OrderedPair = OrderedPair(1,1), collector: (Instant) -> Unit = {}): Int? {
    var instant = this
    var towards = towards(target, drag)
    var steps = 0
    while (towards == true) {
        instant = instant.next(drag)
        towards = instant.towards(target, drag)
        collector(instant)
        steps++
    }

    return if (towards == null) steps else null
}

fun OrderedPair.xVelocityHits(target: Target): List<Int> =
    (x..target.horizontal.endInclusive).fold(mutableListOf<Int>()) { acc, velocity ->
        Instant(x to target.vertical.start, velocity to 0).hits(target, drag = OrderedPair(1, 0))
            ?.let { acc.add(velocity) }
        acc
    }

fun OrderedPair.hits(target: Target): List<Instant> {
    return xVelocityHits(target).fold(mutableListOf<Instant>()) { acc, xVelocity ->
        (target.vertical.start..-(target.vertical.start)).forEach { yVelocity -> // lol lazy
            Instant(x to y, xVelocity to yVelocity).let { instant ->
                instant.hits(target)?.let { acc.add(instant) } 
            }
        }
        acc
    }
}

fun Instant.maxHeight(target: Target): Int? {
    var maxHeight = point.y
    return hits(target) { maxHeight = maxOf(it.point.y, maxHeight) }
        ?.let { maxHeight } 
}

OrderedPair(0, 0).hits(targets[0]).mapNotNull { it.maxHeight(targets[0]) }.maxOrNull()

13041

### Notes

Not sure if this is the best way, but tried to figure out every valid velocity that hits the target by calculating all valid x velocities, all valid y velocities and taking their product. Once we have each valid starting velocity, we can compare the max height they each reach and take the largest value out of all of them.

To calculate the possible x values I checked each x velocity from the start to the target hit the target's x coordinates.
To calculate the possible y values I checked each y velocity between the target's y starting coordinates and its inverse.

## Part 2

How many distinct initial `Instant::velocity` values hit the `target`?

In [303]:
OrderedPair(0,0).hits(targets[0]).size

1031

### Notes

I was fortunate enough that the work in part 1 already told me this value.